# Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
import snscrape.modules.twitter as sntwitter
from better_profanity import profanity
from textblob import TextBlob 
import nltk
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
filter_name = 'weather since:2021-02-24 until:2023-02-21 exclude:retweets exclude:replies lang:en'

scraper = sntwitter.TwitterSearchScraper(filter_name)
scraper
tweets = []
n_tweets = 100
for i, tweet in tqdm(enumerate(scraper.get_items()), total = n_tweets):
    data= [
        tweet.id,
        tweet.date,
        tweet.rawContent,
        tweet.user.username,
        tweet.likeCount,
        tweet.retweetCount,
        tweet.replyCount,
        tweet.lang,
        tweet.user.location
    ]
#     if tweet.lang == 'en':
    tweets.append(data)
    if i > n_tweets:
        break
tweet_df2 = pd.DataFrame(tweets, columns = ['id','datetime','rawcontent','username','likeCount','retweetCount','replyCount','language','location'])

  0%|          | 0/100 [00:00<?, ?it/s]

In [3]:
tweet_df2

,id,datetime,rawcontent,username,likeCount,retweetCount,replyCount,language,location
0,1627820372698562570,2023-02-20 23:59:54+00:00,"Sunset for February 20, 2023 at 05:31PM! Have ...",GallatinWeather,0,0,0,en,
1,1627820366725894144,2023-02-20 23:59:53+00:00,"Report at 00:00 \rWind:SW at av. 0mph, max gus...",HBayWeather,0,0,0,en,Sunny Herne Bay (the UK one)
2,1627820355183075328,2023-02-20 23:59:50+00:00,I think I froze half my piercings off in this ...,gymlifeanimal,200,3,37,en,On a beach somewhere
3,1627820344923783171,2023-02-20 23:59:48+00:00,Your weather for tomorrow @denverchannel https...,MikeNelson247,17,1,1,en,"Denver, CO"
4,1627820342084268034,2023-02-20 23:59:47+00:00,"9 years ago today in Breckinridge, Colorado! I...",GamecockSmoke,1,0,0,en,
...,...,...,...,...,...,...,...,...,...
97,1627819146028146693,2023-02-20 23:55:02+00:00,"Wichita Weather 61F at 5:55 PM, Wind WNW 1 Gus...",SouthWindWX,0,0,0,en,
98,1627819141544419331,2023-02-20 23:55:01+00:00,"Dixon, MO At: 5:55 PM Temp: 57.4°F Visibility:...",dixonweather,0,0,0,en,"Dixon, Missouri USA"
99,1627819138252001280,2023-02-20 23:55:00+00:00,"Wind 5 mph ENE. Barometer 1017.71 hPa, Falling...",quarmbyweather,0,0,0,en,Huddersfield
100,1627819128370110465,2023-02-20 23:54:58+00:00,There is a Fire Weather Watch on Tuesday and a...,KCBD11,0,0,0,en,"Lubbock, TX"
